In [1]:
import pandas as pd
import requests
import json
from datetime import date, timedelta, datetime
from dateutil import parser
import sys
import pandas

In [2]:
study_id = 'StudyHack'
ae_domain_id ='ae'
cm_domain_id = 'cm'
base_url = 'https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/'

In [3]:
ae_subject_id_url = base_url + study_id+'/'+ ae_domain_id+'/subject/list'
cm_subject_id_url = base_url + study_id+'/'+ cm_domain_id+'/subject/list'

In [4]:
try:
    ae_response_data1 = requests.get(ae_subject_id_url).text
    ae_response_value1 = json.loads(ae_response_data1)
    ae_subject_ids = ae_response_value1['data']
except:
    sys.exit("Unable to get the values!")

In [5]:
try:
    cm_response_data1 = requests.get(cm_subject_id_url).text
    cm_response_value1 = json.loads(cm_response_data1)
    cm_subject_ids = cm_response_value1['data']
except:
    sys.exit("Unable to get the values!")

In [ ]:
discrepancy_master_list = []


# iterate through ae_subject_ids 

for subject in ae_subject_ids:
    ae_subject_data_url = base_url + "/" + study_id+"/" + ae_domain_id + "/subject/" + str(subject)+ "/list"
    ae_response_data2 = requests.get(ae_subject_data_url).text
    ae_response_value2 = json.loads(ae_response_data2)
    ae_subject_data = ae_response_value2['data']
    
    cm_subject_data_url = base_url + "/" + study_id+"/" + cm_domain_id + "/subject/" + str(subject)+ "/list"
    cm_response_data2 = requests.get(cm_subject_data_url).text
    cm_response_value2 = json.loads(cm_response_data2)
    cm_subject_data = cm_response_value2['data']
    
    
    discrepancy_list = type1_check(ae_subject_data,cm_subject_data)
    #check whether discrepancy list is not empty then add it to discrepancy_master_list
    

    discrepancy_list = type2_check(ae_subject_data,cm_subject_data)
    #check whether discrepancy list is not empty then add it to discrepancy_master_list
    
    discrepancy_list = type3_check(ae_subject_data)
    #check whether discrepancy list is not empty then add it to discrepancy_master_list
    
    
    discrepancy_list = type4_check(cm_subject_data)
    #check whether discrepancy list is not empty then add it to discrepancy_master_list
    
    
# To submit discrepancies    
submission_url = base_url + study_id
email_address = 'umatti97@gmail.com'

# iterate through every discrepancy in discrepancy_master_list

for discrepancy in discrepancy_master_list:
    discrepancy["email_address"] = email_address
    try:
        r = requests.post(submission_url,data=discrepancy)
    except:
        sys.exit("Unable to submit discrepancy")

In [7]:
# Patient and row for which Medication is given prior to Adverse Event
# Function will get data at subject_id level 
# Identify discrepancy and return discrepancy_list as a list 

def type1_check(ae_subject_data,cm_subject_data):
    discrepancy_list = []
    

    #Iterate through cm_subject_data
    for cm_row in cm_subject_data:
        
        #fetch cmaeno, cmstdat,cmendat, cmtrt from cm_subject_data
        cmaeno = cm_row['cmaeno']
        cmstdat = cm_row['cmstdat']
        cmendat = cm_row['cmendat']
        cmtrt = cm_row['cmtrt']
        
        #Iterate through ae_subject_data
        for ae_row in ae_subject_data:
            
            #fetch aespid, aestdat, aeendat, aeterm from ae_subject_data
            aespid = ae_row['aespid']
            aestdat = ae_row['aestdat']
            aeendat = ae_row['aeendat']
            aeterm = ae_row['aeterm']
            
            #Check whether aespid and cm_ae_no are equal
            if str(aespid) == cmaeno:
                #Check whether cmstdat is lesser than aestdat
                if cmstdat < aestdat:
                    #For the records satisfying condition prepare payload
                    payload={}
                    payload['formname'] = cm_row['formname']
                    payload['formid'] = cm_row['formid']
                    payload['formidx']= cm_row['formidx']
                    payload['type'] = 'TYPE1'
                    payload['subjectid'] = cm_row['subjid']
                    discrepancy_list.append(payload)
            
    return discrepancy_list

In [8]:
dl1 = type1_check(ae_subject_data,cm_subject_data)

In [9]:
#Type 2 check : Patients and rows for which days Medications are given and Adverse Event occur don't match

# Patients and rows for which days Medications are given and Adverse Event occur don't match
# Function will get data at subject_id level 
# Identify discrepancy and return discrepancy_list as a list 

def type2_check(ae_subject_data,cm_subject_data):
    discrepancy_list = []
    

    #Iterate through cm_subject_data
    for cm_row in cm_subject_data:
        
        #fetch cmaeno, cmstdat,cmendat, cmtrt from cm_subject_data
        cmaeno = cm_row['cmaeno']
        cmstdat = cm_row['cmstdat']
        cmendat = cm_row['cmendat']
        cmtrt = cm_row['cmtrt']
        
        #Iterate through ae_subject_data
        for ae_row in ae_subject_data:
            
            #fetch aespid, aestdat, aeendat, aeterm from ae_subject_data
            aespid = ae_row['aespid']
            aestdat = ae_row['aestdat']
            aeendat = ae_row['aeendat']
            aeterm = ae_row['aeterm']
            
            #Check whether aespid and cm_ae_no are equal
            if str(aespid) == cmaeno:
                #Check whether aeendat is lesser than cmstdat
                if aeendat < cmstdat:
                    #For the records satisfying condition prepare payload
                    payload={}
                    payload['formname'] = cm_row['formname']
                    payload['formid'] = cm_row['formid']
                    payload['formidx']= cm_row['formidx']
                    payload['type'] = 'TYPE2'
                    payload['subjectid'] = cm_row['subjid']
                    discrepancy_list.append(payload)
            
    return discrepancy_list

In [10]:
dl2 = type2_check(ae_subject_data,cm_subject_data)

In [11]:
# Write a method to calculate the dates between two given dates 
# Return a list of dates in between two given dates
# for e.g. 01/01/2021 and 01/05/2021 are the two dates. Function should return 
# 01/01/2021,01/02/2021,01/03/2021,01/04/2021,01/05/2021
def func_inbetween_dates(start_date,end_date):
    start = parser.parse(start_date)
    end = parser.parse(end_date)
    
    delta = end - start
    
    inbetween_dates=[]
    
    for i in range(delta.days + 1):
        days = (start+ timedelta(days=i)).strftime('%d-%m-%y')
        day = datetime.strptime(days, '%d-%m-%y').date()
        inbetween_dates.append(day)


    return inbetween_dates

In [12]:
#Type 3 check : Duplicate Adverse events are entered or Adverse Events overlap. 

def type3_check(ae_subject_data):
    discrepancy_list = []
    
    #Iterate through ae_subject_data
    
    for ae_row in ae_subject_data:
        
        #fetch aespid,aestdat,aeendat,aeterm 
        aespid = ae_row['aespid']
        aestdat = ae_row['aestdat']
        aeendat = ae_row['aeendat']
        aeterm = ae_row['aeterm']
        
        #filtering the invalid dates
        if aeendat == '':
            aeendat = 'invalid'
        else:
            aeendat_ = aeendat
        
        #call func_inbetween_dates method and get the list of dates in between aestdat,aeendat.
        
        inbetween_dates = func_inbetween_dates(aestdat,aeendat_)

        
        #Nested for loop on ae_subject_data 
        for ae_row1 in ae_subject_data:
            
            #fetch aespid,aestdat,aeendat,aeterm and assign it to different variables 
            aespid1 = ae_row1['aespid']
            aestdat1 = ae_row1['aestdat']
            aeendat1 = ae_row1['aeendat']
            aeterm1 = ae_row1['aeterm']
            
            #filtering the invalid dates
            if aeendat1 == '':
                aeendat1 = 'invalid'
            else:
                aeendat1_ = aeendat1
            
            d1 = datetime.strptime(aestdat1, '%d-%b-%y').date()
            d2 = datetime.strptime(aeendat1_, '%d-%b-%y').date()
            
            # check to make sure that you are not looking at the same row using aespid
            if aespid != aespid1:
            # Check to make sure aeterm between loop1 and loop2 are matching.
                if aeterm == aeterm1:
            # check to see whether aestdat from loop2 is in inbetween_dates or aeendat from loop2 is in inbetween_dates
                    if (d1 in inbetween_dates) or (d2 in inbetween_dates):    
                        payload={}
                        payload['formname'] = ae_row['formname']
                        payload['formid'] = ae_row['formid']
                        payload['formidx']= ae_row['formidx']
                        payload['type'] = 'TYPE3'
                        payload['subjectid'] = ae_row['subjid']
                        discrepancy_list.append(payload)
    return discrepancy_list
    
                        
    

In [13]:
dl3 = type3_check(ae_subject_data)

In [14]:
#Type 4 check : Duplicate Concomitant Medications or overlap.

def type4_check(cm_subject_data):
    
    discrepancy_list = []
    
    #Iterate through cm_subject_data
    
    for cm_row in cm_subject_data:
        
        #fetch cmaeno,cmstdat,cmendat,cmtrt 
        cmaeno = cm_row['cmaeno']
        cmstdat = cm_row['cmstdat']
        cmendat = cm_row['cmendat']
        cmtrt = cm_row['cmtrt']
        
        #filtering invalid dates
        d1 = cmstdat.split('/')
        if (len(d1) > 1) or (cmstdat == ''):
            d1 = cmstdat = 'invalid' 
        else:
            cmstdat_ = cmstdat
        
        d = cmendat.split('/')
        if (len(d) > 1) or (cmendat == ''):
            d = cmendat = 'invalid'
        else:
            cmendat_ = cmendat
            
        #call func_inbetween_dates method and get the list of dates in between cmstdat,cmendat
        
               
        inbetween_dates = func_inbetween_dates(cmstdat_, cmendat_)

        #Nested for loop on cm_subject_data
        
        for cm_row1 in cm_subject_data:
        
            #fetch cmaeno,cmstdat,cmendat,cmtrt  and assign it to different variables 
            cmaeno1 = cm_row1['cmaeno']
            cmstdat1 = cm_row1['cmstdat']
            cmendat1 = cm_row1['cmendat']
            cmtrt1 = cm_row1['cmtrt']
            
            #filtering the invalid dates
            d3 = cmstdat1.split('/')
            if (len(d3) > 1) or (cmstdat1 == ''):
                d3 = cmstdat1 = 'invalid'
            else:
                cmstdat1_ = cmstdat1
            
            d2 = cmendat1.split('/')
            if (len(d2) > 1) or (cmendat1 == ''):
                d2 = cmendat1 = 'invalid'
            else:
                cmendat1_ = cmendat1
    
            
            d1 = datetime.strptime(cmstdat1_, '%d-%b-%y').date()
            d2 = datetime.strptime(cmendat1_, '%d-%b-%y').date()
            # check to make sure that you are not looking at the same row using cmaeno
            if cmaeno != cmaeno1:
            # Check to make sure cmtrt between loop1 and loop2 are matching.
                if cmtrt == cmtrt1:
            # check to see whether cmstdat from loop2 is in inbetween_dates or cmendat from loop2 is in inbetween_dates
                    if (d1 in inbetween_dates) or (d2 in inbetween_dates):
                        payload ={}
                        payload['formname'] = cm_row['formname']
                        payload['formid'] = cm_row['formid']
                        payload['formidx']= cm_row['formidx']
                        payload['type'] = 'TYPE4'
                        payload['subjectid'] = cm_row['subjid']
                        discrepancy_list.append(payload)
                        

    return discrepancy_list

In [15]:
dl4 = type4_check(cm_subject_data)

In [16]:
discrepancy_master_list.append(dl1)
discrepancy_master_list.append(dl2)
discrepancy_master_list.append(dl3)
discrepancy_master_list.append(dl4)